In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-02-01'
final_date = '2022-02-28'
query_error_year = 2022
query_error_month = 2

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\02. FEBRERO\DATA\ERROR FEBRERO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 10)
#                 & (query_error['year'] == 2021)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15 22:01:46,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15 22:02:08,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01 19:53:09,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12 17:07:39,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12 17:07:25,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(70433, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_10908/3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
61776,61776,b1931e77-d8e5-4ced-84f7-64250dc488e5,Super Abarrotes Maythe Guadalupe,MDVCSQB98,2022-01-30,92.95,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:07,2,2022,NEW,ERROR_USUAL,b1931e77-d8e5-4ced-84f7-64250dc488e5,2022-02-01,1,Actual
61732,61732,f0eec155-ecb6-4e5e-b943-40c3a03d156f,Centro de Especialidad en Piano,3NHKVGQLY,2022-01-30,"2,108.14",ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-02-01 19:09:59,2,2022,NEW,ERROR_USUAL,f0eec155-ecb6-4e5e-b943-40c3a03d156f,2022-02-01,1,Actual
61735,61735,0de300fc-5b25-453d-b9d5-3cbdd4711522,Tuulbar,QNZQVRR9L,2022-01-30,"1,199.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:08,2,2022,NEW,ERROR_USUAL,0de300fc-5b25-453d-b9d5-3cbdd4711522,2022-02-01,1,Actual
61736,61736,f8a2ed1f-26ae-47b6-9595-3ff0e4ba4440,NOVEDADES Y EXCLUSIVIDADES JRO,YXPH5H9CY,2022-01-30,642.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-02-01 19:10:03,2,2022,NEW,ERROR_USUAL,f8a2ed1f-26ae-47b6-9595-3ff0e4ba4440,2022-02-01,1,Actual
61737,61737,66dd31c6-c0d7-4f4b-9f94-1fe0e9e58aaf,Supercyto,K54RHCJNX,2022-01-30,105.41,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:07,2,2022,NEW,ERROR_USUAL,66dd31c6-c0d7-4f4b-9f94-1fe0e9e58aaf,2022-02-01,1,Actual


In [9]:
query_error_current_month.shape

(8858, 17)

In [10]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [11]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [12]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [13]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_10908/3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
46834,46834,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,GPRC4FTNP,2021-11-24,"2,416.58",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:24,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN
47001,47001,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,7YWSWSV5C,2021-11-25,"1,959.60",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:24,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN
47193,47193,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,GDPSR3MT8,2021-11-26,448.47,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:22,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN
47530,47530,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,B7XDQJXVX,2021-11-27,229.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:23,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN


In [14]:
query_new_last_month.shape

(4, 17)

# INFORMACIÓN QUERIES - PAGOS

In [15]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [16]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [17]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [18]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [19]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\02. FEBRUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\02. FEBRERO\QUERIES\*.csv', recursive=False)

In [20]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [21]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [22]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\01. JANUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\01. ENERO\QUERIES\*.csv', recursive=False)

In [23]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [24]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

## ALL FILES

In [25]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,file_name,date_query,tipo_dispersion
1387,052e5a13-0e2f-4467-a58a-3d0cf9062c84,Joojeans,nimbe.candlo@gmail.com,222VCCTSR,2022-01-01,2.00,SANTANDER,2022-01-01 08:13:57,Nimbe yazmin guzman candelario,'014180606134503330,2022-01-01 08:03:37,"Nimbe yazmin guzman candelario,,2.00,014180606...",BNTE 2022-01-01.csv,2022-01-01,TRAD
16151,3d024780-d673-4a93-80df-1a222e4b6c09,EL CHINGON,superdeposito.elchingon@gmail.com,222XZG3T8,2022-01-01,3.20,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-01 08:14:31,Juan Carlos Ocotoxtle Ecatl,'659802020000124489,2022-01-01 08:05:50,"Juan Carlos Ocotoxtle Ecatl,,3.20,659802020000...",BNTE 2022-01-01.csv,2022-01-01,TRAD
4829,122f3e4e-961f-4647-90a0-470299135b21,Jeans y lencera ZIBA,cruz2789@hotmail.com,223VZ9CQN,2022-01-01,"2,119.62","BBVA Bancomer, S.A.",2022-01-01 08:15:03,Samuel Cruz Romero,'012045015049566792,2022-01-01 08:04:14,"Samuel Cruz Romero,,2119.62,012045015049566792...",BNTE 2022-01-01.csv,2022-01-01,TRAD
45918,ad73cff5-ceb6-44bc-ac5d-4a0c102ade46,Panaderia Mr Clark Morelos,ventasmorelosmr@gmail.com,223YBPZ5Q,2022-01-01,"2,724.26",INBURSA,2022-01-01 08:14:31,Javier Ernesto Hernandez Romero,'036760500592593444,2022-01-01 08:10:43,"Javier Ernesto Hernandez Romero,,2724.26,03676...",BNTE 2022-01-01.csv,2022-01-01,TRAD
25530,60f936b7-15e4-4e37-85c7-5960b82f62ba,Angeles Reyes garcia,emmanueldlgm@gmail.com,224MTQXW9,2022-01-01,"4,185.90","Scotiabank Inverlat, S.A.",2022-01-01 08:14:31,Emmanuel De la garza Moranchel,'044650256014193503,2022-01-01 08:07:30,"Emmanuel De la garza Moranchel,,4185.90,044650...",BNTE 2022-01-01.csv,2022-01-01,TRAD


In [26]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [27]:
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,file_name,date_query,tipo_dispersion,tipo_pago
1387,052e5a13-0e2f-4467-a58a-3d0cf9062c84,Joojeans,nimbe.candlo@gmail.com,222VCCTSR,2022-01-01,2.00,SANTANDER,2022-01-01 08:13:57,Nimbe yazmin guzman candelario,'014180606134503330,2022-01-01 08:03:37,"Nimbe yazmin guzman candelario,,2.00,014180606...",BNTE 2022-01-01.csv,2022-01-01,TRAD,Pago del día
16151,3d024780-d673-4a93-80df-1a222e4b6c09,EL CHINGON,superdeposito.elchingon@gmail.com,222XZG3T8,2022-01-01,3.20,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-01 08:14:31,Juan Carlos Ocotoxtle Ecatl,'659802020000124489,2022-01-01 08:05:50,"Juan Carlos Ocotoxtle Ecatl,,3.20,659802020000...",BNTE 2022-01-01.csv,2022-01-01,TRAD,Pago del día
4829,122f3e4e-961f-4647-90a0-470299135b21,Jeans y lencera ZIBA,cruz2789@hotmail.com,223VZ9CQN,2022-01-01,"2,119.62","BBVA Bancomer, S.A.",2022-01-01 08:15:03,Samuel Cruz Romero,'012045015049566792,2022-01-01 08:04:14,"Samuel Cruz Romero,,2119.62,012045015049566792...",BNTE 2022-01-01.csv,2022-01-01,TRAD,Pago del día
45918,ad73cff5-ceb6-44bc-ac5d-4a0c102ade46,Panaderia Mr Clark Morelos,ventasmorelosmr@gmail.com,223YBPZ5Q,2022-01-01,"2,724.26",INBURSA,2022-01-01 08:14:31,Javier Ernesto Hernandez Romero,'036760500592593444,2022-01-01 08:10:43,"Javier Ernesto Hernandez Romero,,2724.26,03676...",BNTE 2022-01-01.csv,2022-01-01,TRAD,Pago del día
25530,60f936b7-15e4-4e37-85c7-5960b82f62ba,Angeles Reyes garcia,emmanueldlgm@gmail.com,224MTQXW9,2022-01-01,"4,185.90","Scotiabank Inverlat, S.A.",2022-01-01 08:14:31,Emmanuel De la garza Moranchel,'044650256014193503,2022-01-01 08:07:30,"Emmanuel De la garza Moranchel,,4185.90,044650...",BNTE 2022-01-01.csv,2022-01-01,TRAD,Pago del día


In [28]:
allfiles.shape

(4613049, 16)

# CRUCE 1 - MES ACTUAL

In [29]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,052e5a13-0e2f-4467-a58a-3d0cf9062c84,Joojeans,nimbe.candlo@gmail.com,222VCCTSR,2022-01-01,2.00,SANTANDER,2022-01-01 08:13:57,Nimbe yazmin guzman candelario,'014180606134503330,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,3d024780-d673-4a93-80df-1a222e4b6c09,EL CHINGON,superdeposito.elchingon@gmail.com,222XZG3T8,2022-01-01,3.20,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-01 08:14:31,Juan Carlos Ocotoxtle Ecatl,'659802020000124489,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,122f3e4e-961f-4647-90a0-470299135b21,Jeans y lencera ZIBA,cruz2789@hotmail.com,223VZ9CQN,2022-01-01,"2,119.62","BBVA Bancomer, S.A.",2022-01-01 08:15:03,Samuel Cruz Romero,'012045015049566792,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,ad73cff5-ceb6-44bc-ac5d-4a0c102ade46,Panaderia Mr Clark Morelos,ventasmorelosmr@gmail.com,223YBPZ5Q,2022-01-01,"2,724.26",INBURSA,2022-01-01 08:14:31,Javier Ernesto Hernandez Romero,'036760500592593444,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,60f936b7-15e4-4e37-85c7-5960b82f62ba,Angeles Reyes garcia,emmanueldlgm@gmail.com,224MTQXW9,2022-01-01,"4,185.90","Scotiabank Inverlat, S.A.",2022-01-01 08:14:31,Emmanuel De la garza Moranchel,'044650256014193503,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [30]:
cruce_1.shape

(4613049, 24)

In [31]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_10908/1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2066500,d41331f3-52b4-4437-a2ec-5ed2317c7327,Tacos Wendy,tacoswendymty@gmail.com,22RLBBPKF,2022-01-29,560.58,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-29 10:42:28,Miguel Angel Lopez Ruiz,'659802020000540186,...,TRAD,Pago del día,560.58,2022-02-01 19:09:52,2022-02-01,2.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
2068666,40988ad1-d794-406d-b934-9629e9294dc3,PIZZERIA MAX,abrahamuicabperez@gmail.com,2R7G4MWQ3,2022-01-29,285.89,ASP INTEGRA OPC,2022-01-29 10:27:48,Jesús Abraham Uicab Pérez,'659802020000001492,...,TRAD,Pago del día,285.89,2022-02-01 19:09:53,2022-02-01,2.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
2068767,48374596-32ad-4e03-9604-344b9c952d62,Abarrotes Paty,alondraanahiblancovazquez3@gmail.com,2S8247SNK,2022-01-29,179.19,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-29 16:41:45,Norma Patricia Vazquez Garcia,'659802020000282952,...,TRAD,Pago del día,179.19,2022-02-01 19:09:55,2022-02-01,2.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
2068800,01a41d3b-a24a-4e71-be7e-a8790e35628a,Sushi Manía,angelaapostolgarcia10@gmail.com,2SK99MXRD,2022-01-29,"1,082.81","Scotiabank Inverlat, S.A.",2022-01-29 11:02:22,Israel Apostol Arango,'044225256039155647,...,TRAD,Pago del día,"1,082.81",2022-02-01 19:09:44,2022-02-01,2.00,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,ERROR_USUAL,NEW
2069760,47f85213-6580-41fe-a971-3f90309cfc1b,La montaña,ismael.ubilla@gmail.com,34MFDF4Q6,2022-01-29,389.04,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-29 10:28:07,Ismael Ubilla Escamilla,'659802020000519010,...,TRAD,Pago del día,389.04,2022-02-01 19:09:53,2022-02-01,2.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW


In [32]:
cruce_1_final.shape

(3683, 24)

In [33]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [34]:
duplicates_cruce_1_final.shape

(0, 24)

# CRUCE 2 MES ACTUAL

In [35]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,61776,b1931e77-d8e5-4ced-84f7-64250dc488e5,Super Abarrotes Maythe Guadalupe,MDVCSQB98,2022-01-30,92.95,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:07,2,2022,NEW,ERROR_USUAL,b1931e77-d8e5-4ced-84f7-64250dc488e5,2022-02-01,1,Actual,TRAD,Pago del día
1,61732,f0eec155-ecb6-4e5e-b943-40c3a03d156f,Centro de Especialidad en Piano,3NHKVGQLY,2022-01-30,"2,108.14",ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-02-01 19:09:59,2,2022,NEW,ERROR_USUAL,f0eec155-ecb6-4e5e-b943-40c3a03d156f,2022-02-01,1,Actual,TRAD,Pago del día
2,61735,0de300fc-5b25-453d-b9d5-3cbdd4711522,Tuulbar,QNZQVRR9L,2022-01-30,"1,199.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:08,2,2022,NEW,ERROR_USUAL,0de300fc-5b25-453d-b9d5-3cbdd4711522,2022-02-01,1,Actual,TRAD,Pago del día
3,61736,f8a2ed1f-26ae-47b6-9595-3ff0e4ba4440,NOVEDADES Y EXCLUSIVIDADES JRO,YXPH5H9CY,2022-01-30,642.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-02-01 19:10:03,2,2022,NEW,ERROR_USUAL,f8a2ed1f-26ae-47b6-9595-3ff0e4ba4440,2022-02-01,1,Actual,TRAD,Pago del día
4,61737,66dd31c6-c0d7-4f4b-9f94-1fe0e9e58aaf,Supercyto,K54RHCJNX,2022-01-30,105.41,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:07,2,2022,NEW,ERROR_USUAL,66dd31c6-c0d7-4f4b-9f94-1fe0e9e58aaf,2022-02-01,1,Actual,TRAD,Pago del día


In [36]:
cruce_2.shape

(9815, 19)

In [37]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,61776,b1931e77-d8e5-4ced-84f7-64250dc488e5,Super Abarrotes Maythe Guadalupe,MDVCSQB98,2022-01-30,92.95,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:07,2,2022,NEW,ERROR_USUAL,b1931e77-d8e5-4ced-84f7-64250dc488e5,2022-02-01,1,Actual,TRAD,Pago del día
1,61732,f0eec155-ecb6-4e5e-b943-40c3a03d156f,Centro de Especialidad en Piano,3NHKVGQLY,2022-01-30,"2,108.14",ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-02-01 19:09:59,2,2022,NEW,ERROR_USUAL,f0eec155-ecb6-4e5e-b943-40c3a03d156f,2022-02-01,1,Actual,TRAD,Pago del día
2,61735,0de300fc-5b25-453d-b9d5-3cbdd4711522,Tuulbar,QNZQVRR9L,2022-01-30,"1,199.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:08,2,2022,NEW,ERROR_USUAL,0de300fc-5b25-453d-b9d5-3cbdd4711522,2022-02-01,1,Actual,TRAD,Pago del día
3,61736,f8a2ed1f-26ae-47b6-9595-3ff0e4ba4440,NOVEDADES Y EXCLUSIVIDADES JRO,YXPH5H9CY,2022-01-30,642.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-02-01 19:10:03,2,2022,NEW,ERROR_USUAL,f8a2ed1f-26ae-47b6-9595-3ff0e4ba4440,2022-02-01,1,Actual,TRAD,Pago del día
4,61737,66dd31c6-c0d7-4f4b-9f94-1fe0e9e58aaf,Supercyto,K54RHCJNX,2022-01-30,105.41,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-02-01 19:10:07,2,2022,NEW,ERROR_USUAL,66dd31c6-c0d7-4f4b-9f94-1fe0e9e58aaf,2022-02-01,1,Actual,TRAD,Pago del día


In [38]:
cruce_2_final.shape

(4640, 19)

In [39]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
309,62006,6a3d068b-4eb4-4970-be29-347e723fe70f,LA CHONITA,JVY2J25NF,2022-01-31,"1,067.81",ERROR,Excede el limite de saldo autorizado de la cue...,2022-02-01 13:51:03,2,2022,NEW,ERROR_USUAL,6a3d068b-4eb4-4970-be29-347e723fe70f,2022-02-01,1,Actual,TRAD,Pago del día
310,62006,6a3d068b-4eb4-4970-be29-347e723fe70f,LA CHONITA,JVY2J25NF,2022-01-31,"1,067.81",ERROR,Excede el limite de saldo autorizado de la cue...,2022-02-01 13:51:03,2,2022,NEW,ERROR_USUAL,6a3d068b-4eb4-4970-be29-347e723fe70f,2022-02-01,1,Actual,TRAD,Reprogramación
484,50966,6a3d068b-4eb4-4970-be29-347e723fe70f,LA CHONITA,KMVXH23J2,2021-12-14,483.81,ERROR,Excede el limite de saldo autorizado de la cue...,2022-02-01 13:35:38,2,2022,ERROR,ERROR_USUAL,6a3d068b-4eb4-4970-be29-347e723fe70f,2022-02-01,1,Actual,TRAD,Reprogramación
485,50966,6a3d068b-4eb4-4970-be29-347e723fe70f,LA CHONITA,KMVXH23J2,2021-12-14,483.81,ERROR,Excede el limite de saldo autorizado de la cue...,2022-02-01 13:35:38,2,2022,ERROR,ERROR_USUAL,6a3d068b-4eb4-4970-be29-347e723fe70f,2022-02-01,1,Actual,TRAD,Reprogramación
486,58367,ae868353-ad64-43b7-aeb2-8a2058b123bf,Calzados Viany,PFT57JW56,2022-01-17,2.00,ERROR,Tipo de operacion erronea,2022-02-01 13:37:43,2,2022,NEW,ERROR_USUAL,ae868353-ad64-43b7-aeb2-8a2058b123bf,2022-02-01,1,Actual,TRAD,Pago del día


In [40]:
duplicates_cruce_2_final.shape

(1399, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [41]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-02-01           0.00      39,452.89   
1                         2022-02-02       1,406.69      11,331.16   
2                         2022-02-03           0.00         161.51   
3                         2022-02-04           0.00           0.00   
4                         2022-02-05           0.00           0.00   

                                                                     
esquema_query                                                   All  
status_cubeta            NEW                                         
tipo_dispersion           FP         TRAD                            
tipo_pago       Pago del día Pago del día Reprogramación             
0                     115.95   439,280.63      19,098.72 497,948.19  
1                  20,504.08    23,500.26           0.00  56,742.19  
2                     388.09    29,042.39       4,408.52  34,000.51  
3                   1,045.44    24,591.29       3,046.50  28,683.23  
4                       0.00     3,310.15         481.99   3,792.14

In [42]:
group_tipo_pago.shape

(29, 7)

In [43]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-02-01      39,452.89   439,396.58      19,098.72   
1                       2022-02-02      12,737.85    44,004.34           0.00   
2                       2022-02-03         161.51    29,430.48       4,408.52   
3                       2022-02-04           0.00    25,636.73       3,046.50   
4                       2022-02-05           0.00     3,310.15         481.99   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             497,948.19  
1              56,742.19  
2              34,000.51  
3              28,683.23  
4               3,792.14

In [44]:
group_esquema.shape

(29, 5)

# CRUCE 3 - MES ANTERIOR

In [45]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,052e5a13-0e2f-4467-a58a-3d0cf9062c84,Joojeans,nimbe.candlo@gmail.com,222VCCTSR,2022-01-01,2.00,SANTANDER,2022-01-01 08:13:57,Nimbe yazmin guzman candelario,'014180606134503330,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,3d024780-d673-4a93-80df-1a222e4b6c09,EL CHINGON,superdeposito.elchingon@gmail.com,222XZG3T8,2022-01-01,3.20,"Opciones Empresariales del Noreste, S.A. de C....",2022-01-01 08:14:31,Juan Carlos Ocotoxtle Ecatl,'659802020000124489,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,122f3e4e-961f-4647-90a0-470299135b21,Jeans y lencera ZIBA,cruz2789@hotmail.com,223VZ9CQN,2022-01-01,"2,119.62","BBVA Bancomer, S.A.",2022-01-01 08:15:03,Samuel Cruz Romero,'012045015049566792,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,ad73cff5-ceb6-44bc-ac5d-4a0c102ade46,Panaderia Mr Clark Morelos,ventasmorelosmr@gmail.com,223YBPZ5Q,2022-01-01,"2,724.26",INBURSA,2022-01-01 08:14:31,Javier Ernesto Hernandez Romero,'036760500592593444,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,60f936b7-15e4-4e37-85c7-5960b82f62ba,Angeles Reyes garcia,emmanueldlgm@gmail.com,224MTQXW9,2022-01-01,"4,185.90","Scotiabank Inverlat, S.A.",2022-01-01 08:14:31,Emmanuel De la garza Moranchel,'044650256014193503,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [46]:
cruce_3.shape

(4613049, 24)

In [47]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_10908/2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
14376,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,Rosaymenta6320@gmail.com,7YWSWSV5C,2021-11-25,"1,959.60","Banco Nacional de México, S.A.",2022-01-01 08:13:57,bianca janeth cruz cruz,'002790701776963800,...,TRAD,Reprogramación,"1,959.60",2022-01-04 20:03:24,2022-01-04,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
19815,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,Rosaymenta6320@gmail.com,B7XDQJXVX,2021-11-27,229.98,"Banco Nacional de México, S.A.",2022-01-01 08:13:57,bianca janeth cruz cruz,'002790701776963800,...,TRAD,Reprogramación,229.98,2022-01-04 20:03:23,2022-01-04,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
30046,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,Rosaymenta6320@gmail.com,GDPSR3MT8,2021-11-26,448.47,"Banco Nacional de México, S.A.",2022-01-01 08:13:57,bianca janeth cruz cruz,'002790701776963800,...,TRAD,Reprogramación,448.47,2022-01-04 20:03:22,2022-01-04,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
30753,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,Rosaymenta6320@gmail.com,GPRC4FTNP,2021-11-24,"2,416.58","Banco Nacional de México, S.A.",2022-01-01 08:13:57,bianca janeth cruz cruz,'002790701776963800,...,TRAD,Reprogramación,"2,416.58",2022-01-04 20:03:24,2022-01-04,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW


In [48]:
cruce_3_final.shape

(4, 24)

In [49]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [50]:
duplicates_cruce_3_final.shape

(0, 24)

# CRUCE 4 - MES ANTERIOR

In [51]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,46834,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,GPRC4FTNP,2021-11-24,"2,416.58",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:24,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación
1,47001,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,7YWSWSV5C,2021-11-25,"1,959.60",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:24,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación
2,47193,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,GDPSR3MT8,2021-11-26,448.47,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:22,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación
3,47530,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,B7XDQJXVX,2021-11-27,229.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:23,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación


In [52]:
cruce_4.shape

(4, 19)

In [53]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,46834,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,GPRC4FTNP,2021-11-24,"2,416.58",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:24,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación
1,47001,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,7YWSWSV5C,2021-11-25,"1,959.60",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:24,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación
2,47193,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,GDPSR3MT8,2021-11-26,448.47,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:22,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación
3,47530,cbb62319-0f7e-4ce0-849a-36523d3e6581,VERDE MENTA,B7XDQJXVX,2021-11-27,229.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-01-04 20:03:23,1,2022,NEW,ERROR_USUAL,cbb62319-0f7e-4ce0-849a-36523d3e6581,2022-01-04,4,NaN,TRAD,Reprogramación


In [54]:
cruce_4_final.shape

(4, 19)

In [55]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago


In [56]:
duplicates_cruce_4_final.shape

(0, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [57]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x         
esquema_query                           ERROR_USUAL      All
status_cubeta                                   NEW         
tipo_dispersion                                TRAD         
tipo_pago                            Reprogramación         
0                         2022-01-04       5,054.63 5,054.63
1                                All       5,054.63 5,054.63

In [58]:
group_tipo_pago_last_month.shape

(2, 3)

In [59]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount         
esquema_query                         ERROR_USUAL      All
status_cubeta                                 NEW         
tipo_pago                          Reprogramación         
0                       2022-01-04       5,054.63 5,054.63
1                              All       5,054.63 5,054.63

In [60]:
group_esquema_last_month.shape

(2, 3)

In [61]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL FEBRERO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()